Выбор большой языковой модели

In [1]:
from llamba.chatmodels.ollama import OllamaModel
chatbot = OllamaModel(url="http://127.0.0.1:11434/", 
                      endpoint="api/generate", 
                      model="llama3", 
                      num_threads=1, 
                      check_connection_timeout=15, 
                      request_timeout=15)
connection = chatbot.check_connection()
print(connection)

True


Подготовка данных

In [6]:
import torch
from torch import nn
import pandas as pd
import numpy as np

from llamba_library.bioage_model import BioAgeModel
from llamba_library.functions import get_shap_dict
from txai_omics_3.models.tabular.widedeep.ft_transformer \
    import WDFTTransformerModel, FN_SHAP, FN_CHECKPOINT


#### Данные

my_data = {'CXCL9': 2599.629474, 
           'CCL22': 820.306524, 
           'IL6': 0.846377, 
           'PDGFB': 13400.666359, 
           'CD40LG': 1853.847406, 
           'IL27': 1128.886982,
           'VEGFA': 153.574220,
           'CSF1': 239.627236,
           'PDGFA': 1005.844290,
           'CXCL10': 228.229829,
           'Age': 90.454972 }

data = pd.DataFrame(my_data, index=[0])


#### Модель
    
fn_model = FN_CHECKPOINT
model = WDFTTransformerModel.load_from_checkpoint(fn_model)
bioage_model = BioAgeModel(model=model)

def predict_func(x):
    batch = {
        'all': torch.from_numpy(np.float32(x)),
        'continuous': torch.from_numpy(np.float32(x)),
        'categorical': torch.from_numpy(np.int32(x[:, []])),
    }
    return model(batch).cpu().detach().numpy()
shap_dict = get_shap_dict(FN_SHAP)
explainer = shap_dict['explainer']
feats = data.drop(['Age'], axis=1).columns.to_list()

top_n = 3 # количество признаков с наибольшим вкладом

top_shap = bioage_model.get_top_shap(top_n, data, feats, shap_dict) 

h:\Lobachevsky\llamba\llamba_env\Lib\site-packages\pytorch_lightning\utilities\migration\utils.py:55: The loaded checkpoint was produced with Lightning v2.4.0, which is newer than your current Lightning version: v2.1.4
h:\Lobachevsky\llamba\llamba_env\Lib\site-packages\torchmetrics\utilities\prints.py:43: UserWarning: Metric `SpearmanCorrcoef` will save all targets and predictions in the buffer. For large datasets, this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)  # noqa: B028


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu!

Составление запроса для экспертной системы 

In [ ]:
from llamba.connector import LlambaConnector

connector = LlambaConnector(bioage_model=bioage_model, chat_model=chatbot)
prompts = connector.produce_feat_analysis_prompts(top_n=top_n, 
                                     data=top_shap['data'], 
                                     feats=top_shap['feats'], 
                                     values=top_shap['values'])
print("Prompts: ")
for prompt in prompts:
    print(prompt)

NameError: name 'top_shap' is not defined

Передача экспертной системе данных 

In [4]:
res = connector.query_prompts()
print("Analysis: \n")
print(res)

Analysis: 

PDGFB: 13400.666359
CD40LG: 1853.847406
CXCL9: 2599.629474
PDGFB stands for platelet-derived growth factor-B. It's a protein involved in cell proliferation, migration, and differentiation, particularly during embryonic development and wound healing.

An increased level of PDGFB has been observed in various age-related disorders, including age-related macular degeneration, atherosclerosis, and Alzheimer's disease. Elevated levels may contribute to cellular senescence, inflammation, and tissue fibrosis, which can accelerate aging processes and increase the risk of age-related diseases.

CD40LG, also known as TNFSF5 or TR6, is a protein encoded by the CD40LG gene. It is a ligand for the receptor CD40, which plays a key role in the activation and differentiation of immune cells.

An increased level of CD40LG can be indicative of an active immune response, particularly in older adults. Elevated levels have been associated with various age-related diseases, including Alzheimer's 